<a href="https://colab.research.google.com/github/LiangShuLing/TensorFlowLearning/blob/main/jax/jax-ResNet50%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy.random as npr
import jax.numpy as jnp
from jax.experimental import optimizers
from jax.experimental import stax
from jax.experimental.stax import AvgPool,BatchNorm,Dense,FanInSum,FanOut,Flatten, GeneralConv,Identity,MaxPool,Relu,LogSoftmax

jax.experimental.stax里面包含很多函数，大部分都是定义网络的函数，比如Dense,GeneralConv, Conv--实现了GeneralConv;
GeneralConvTranspose, 

池化： AvgPool, MaxPool

BatchNorm层：参数标准化

FanOut(b): fan-out layer,对输入进行泛化处理，整体乘以参数b

FanInSum(): FanIn sum层，对输入进行求和处理

函数：实现了jax.nn里面的函数
Tanh，Relu，Exp，LogSoftmax，Softmax，Softplus，Sigmoid ，Elu，LeakyRelu，Selu，Gelu

shape_dependent(make_layer): 
  """Combinator to delay layer constructor pair until input shapes are known.

In [2]:
#通过Conv层实现ConvBlock
def ConvBlock(kernel_size,filters,strides=(2,2)):
  ks=kernel_size
  filter1,filter2,filter3=filters

  Main=stax.serial(
      Conv(filter1,(1,1),strides),BatchNorm(),Relu,
      Conv(filter2,(ks,ks),padding='SAME'),BatchNorm(),Relu,
      Conv(filter3,(1,1)),BatchNorm())
  #定义网络结构，类似于keras的Sequential容器,下面再定义一个网络结构
  Shortcut=stax.serial(Conv(filter3,(1,1),strides),BatchNorm())
  return stax.serial(FanOut(2),stax.parallel(Main,Shortcut),FanInsum,Relu)  #再次拼接模型并返回



In [3]:
def IdentityBlock(kernel_size,filters):
  ks=kernel_size
  filter1,filter2=filters
  def make_main(input_shape):
    return stax.serial(
        Conv(filter1,(1,1)),BatchNorm(),Relu,
        Conv(filter2,(ks,ks),padding='SAME'),BatchNorm(),Relu,
        Conv(input_shape[3],(1,1)),BatchNorm())
    
    Main=stax.shape_dependent(make_main)   #等到输入确定后再工作的延迟层
    return stax.serial(FanOut(2),stax.parallel(Main,Identity),FanInSum,Relu)


In [5]:
 #num_classes定义输出层多少类
def ResNet50(num_classes): 
  return stax.serial(
      GeneralConv(('HWCN','OIHW','NHWC'),64,(7,7),(2,2),'SAME'),
      BatchNorm(),Relu,MaxPool((3,3),strides=(2,2)),
      ConvBlock(3,[64,64,256],strides=(1,1)),
      IdentityBlock(3, [64, 64]),
      IdentityBlock(3, [64, 64]),
      ConvBlock(3, [128, 128, 512]),
      IdentityBlock(3, [128, 128]),
      IdentityBlock(3, [128, 128]),
      IdentityBlock(3, [128, 128]),
      ConvBlock(3, [256, 256, 1024]),
      IdentityBlock(3, [256, 256]),
      IdentityBlock(3, [256, 256]),
      IdentityBlock(3, [256, 256]),
      IdentityBlock(3, [256, 256]),
      IdentityBlock(3, [256, 256]),
      ConvBlock(3, [512, 512, 2048]),
      IdentityBlock(3, [512, 512]),
      IdentityBlock(3, [512, 512]),
      AvgPool((7, 7)), Flatten, Dense(num_classes), LogSoftmax
  ) 

